In [16]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

import torchvision
from torchvision.models import ResNet50_Weights
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from torchmetrics import Accuracy

import wandb

from typing import Any, Literal
import random

In [3]:
path = os.path.dirname(os.path.realpath('__dir__'))
path = os.path.dirname(path)
train_path = os.path.join(path, 'inaturalist_12K/train/')
test_path = os.path.join(path, 'inaturalist_12K/val/')

In [4]:
def get_datasets(augmentation: bool=True):
    augs = [
            transforms.RandomResizedCrop((224, 224)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            # transforms.RandomRotation(45),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
    ]
    transform = transforms.Compose([
        *random.sample(augs, k=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),
    ] if augmentation else [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),
    ])
    global train_path
    global test_path
    dataset = ImageFolder(train_path, transform=transform)

    train_size = int(0.8 * len(dataset))
    val_size = (len(dataset) - train_size) 
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    test_dataset = ImageFolder(test_path, transform=transform)
    return train_dataset, val_dataset, test_dataset

In [5]:
train_dataset, val_dataset, test_dataset = get_datasets()
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [19]:
class FineTuneModel(pl.LightningModule):
    def __init__(self, lr: float=1e-4) -> None:
        super().__init__()
        self.save_hyperparameters()
        self.lr = lr
        self.backbone = torchvision.models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        for param in self.backbone.parameters():
            param.requires_grad = False
        self.feature_extractor = nn.Sequential(*list(self.backbone.children())[:-1])
        self.classifier = nn.Linear(self.backbone.fc.in_features, 10)
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=10)
        self.train_dataset, self.val_dataset, self.test_dataset = get_datasets()
    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        accuracy = self.accuracy(y_hat, y)
        self.log('train/loss', loss)
        self.log('train/accuracy', accuracy)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        accuracy = self.accuracy(y_hat, y)
        self.log('val/loss', loss)
        self.log('val/accuracy', accuracy)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        accuracy = self.accuracy(y_hat, y)
        self.log('test/loss', loss)
        self.log('test/accuracy', accuracy)
        return {'loss': loss, 'y_hat': y_hat, 'y': y}

    def configure_optimizers(self) -> Any:
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
    
    def train_dataloader(self) -> DataLoader:
        return DataLoader(self.train_dataset, batch_size=32, shuffle=True, num_workers=4)
    
    def test_dataloader(self) -> DataLoader:
        return DataLoader(self.test_dataset, batch_size=32, num_workers=4)
    
    def val_dataloader(self) -> DataLoader:
        return DataLoader(self.val_dataset, batch_size=32, num_workers=4)
        

In [20]:
model = FineTuneModel()

In [21]:
logger = pl.loggers.WandbLogger(name="pretrained-resnet50", project='CS6910-Assignment2')
logger.watch(model)
callbacks = [pl.callbacks.ModelCheckpoint(monitor='val/accuracy', mode='max', save_top_k=1, save_last=True)]
trainer = pl.Trainer(max_epochs=20, devices=-1, logger=logger, callbacks=callbacks, precision='16-mixed')
trainer.fit(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iamunr4v31. Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params
---------------------------------------------------------
0 | backbone          | ResNet             | 25.6 M
1 | feature_extractor | Sequential         | 23.5 M
2 | classifier        | Linear             | 20.5 K
3 | loss_fn           | CrossEntropyLoss   | 0     
4 | accuracy          | MulticlassAccuracy | 0     
---------------------------------------------------------
20.5 K    Trainable params
25.6 M    Non-trainable params
25.6 M    Total params
102.310   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\ashwi\miniconda3\envs\dl_class\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Epoch 9: 100%|██████████| 250/250 [03:32<00:00,  1.18it/s, v_num=8noh]     

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 250/250 [03:35<00:00,  1.16it/s, v_num=8noh]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\ashwi\miniconda3\envs\dl_class\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:478: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 63/63 [00:32<00:00,  1.91it/s]

TypeError: FineTuneModel.on_test_epoch_end() missing 1 required positional argument: 'outputs'

In [22]:
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\ashwi\miniconda3\envs\dl_class\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:478: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 63/63 [06:07<00:00,  5.84s/it]
Testing: 63it [00:34,  1.81it/s]

TypeError: FineTuneModel.on_test_epoch_end() missing 1 required positional argument: 'outputs'